In [1]:
import site
site.addsitedir('../python')  # Always appends to end

import gdal_functions as gd
import ntpath
from osgeo.gdal import GDT_Byte, GDT_Float32, GDT_Int32
import zipfile as zip
import os
from pathlib import Path

In [2]:
# modified so that the PRISM and SWB grids are at a 1km resolution
grid_details = {"xul": -60000., 
"yul": 1735325., 
"rotation": 0.0, 
"proj4_str": "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs", 
"nrow": 1115, 
"ncol": 1110, 
"delr": 1000.0, 
"delc": 1000.0, 
"epsg": 5070}

grd = grid_details

grd['xll'] = grd['xul']
grd['yll'] = grd['yul'] - grd['nrow'] * grd['delr']

grd['xur'] = grd['xul'] + grd['ncol'] * grd['delc']
grd['yur'] = grd['yul']

target_resolution = str(grd['delr']).split('.')[0]

starting_year = 1981
ending_year = 1981

In [3]:
file_prefix = 'map__'
variable_names = ['ppt','tmin', 'tmax']

In [4]:
prism_data_dir = Path('../test_data')
current_dir = Path.cwd()
output_path = current_dir.parent / 'output'
output_path.mkdir(parents=True, exist_ok=True)

In [5]:
# shamelessly stolen from https://stackoverflow.com/questions/8384737/extract-file-name-from-path-no-matter-what-the-os-path-format/8384788
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

# shamelessly stolen from https://stackoverflow.com/questions/18394147/recursive-sub-folder-search-and-return-files-in-a-list-python
prism_filelist = list(Path(prism_data_dir).rglob("*.zip"))

In [8]:
for prism_filename in prism_filelist:
    for variable_name in variable_names:
        prism_file = str(prism_filename)
        # we are only interested in 'zip' files for now
        if str(prism_file).endswith('zip'):
            prism_zip_file = str(prism_file)
            prism_file_basename = path_leaf(prism_zip_file)
            # hacky code to extract the date from the filename
            mylist = prism_file_basename.replace(".","_").split("_")
            list_len = len(mylist)
            param_name = mylist[1]
            file_date = mylist[list_len - 3]
            myyear = int(file_date[0:4])
        
            if myyear >= starting_year and myyear <= ending_year and param_name == variable_name:
                # unzip the *.zip file and do some file renaming
                myzip = zip.ZipFile(str(prism_zip_file))
                mybil=prism_file_basename.replace(".zip",".bil")
                myprj=prism_file_basename.replace(".zip",".prj")
                myhdr=prism_file_basename.replace(".zip",".hdr")
                myzip.extract(mybil)
                myzip.extract(myprj)
                myzip.extract(myhdr)

                # create the new output filename
                prism_output_file_path = output_path / str( file_prefix + param_name + '__' + file_date + '__' + str(target_resolution) + 'm' )
                prism_ascii_output_file = str(prism_output_file_path) + '.asc'
                prism_tiff_output_file = 'PRISM_temp.tif'
                print("  output => ", prism_ascii_output_file)
                # gdalwarp: perform nearest neighbor, resample to swb grid resolution, reproject data
                gd.gdalwarp( src_file=mybil, 
                      dst_file=prism_tiff_output_file,
                      src_proj4=gd.get_proj4(myprj),
                      dst_proj4=grd['proj4_str'],
                      nx=grd['ncol'],
                      ny=grd['nrow'],
                      xll=grd['xll'],
                      yll=grd['yll'],
                      xur=grd['xur'],
                      yur=grd['yur'],
                      output_type=GDT_Float32,
                      resample_algorithm='near')
                # translate the 'tif' file into Arc ASCII
                gd.gdal_translate(src_file=prism_tiff_output_file, dst_file=prism_ascii_output_file)
                # clean up remaining temp files
                os.remove(prism_tiff_output_file)
                os.remove(mybil)
                os.remove(myprj)
                os.remove(myhdr)

  output =>  e:\projects\swb_development\git\swb2_prism\output\map__ppt__19810101__1000m.asc
  output =>  e:\projects\swb_development\git\swb2_prism\output\map__ppt__19810102__1000m.asc
  output =>  e:\projects\swb_development\git\swb2_prism\output\map__tmax__19810101__1000m.asc
  output =>  e:\projects\swb_development\git\swb2_prism\output\map__tmax__19810102__1000m.asc
  output =>  e:\projects\swb_development\git\swb2_prism\output\map__tmin__19810101__1000m.asc
  output =>  e:\projects\swb_development\git\swb2_prism\output\map__tmin__19810102__1000m.asc
